In [4]:
import pandas as pd
import numpy as np

In [5]:
dataset = pd.read_excel("slides.xlsx")

In [6]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,French,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,Thai,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,Burger,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,Thai,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,French,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,Burger,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,Thai,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,Burger,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,"""10 - 30""",0


In [7]:
class node ():
    def __init__(self, gainInfo = None , value  = None, feature = None , right = None , left = None , type = None) :
        self.right = right
        self.left = left
        self.feature = feature
        self.type = type
        self.geinInfo = gainInfo
        self.value = value

In [8]:
class DecisionTree():
    def __init__(self):
        self.root = None
    def build (example , features , parent_examples):
        pass
        
        

In [9]:
def MakeUnique(dataFrame , columnName):
    #print(dataFrame[columnName])
    PossibleValues = pd.unique(dataFrame[columnName])
    #print(PossibleValues)
    indexed = {}
    for count , value in enumerate(PossibleValues):
        indexed[value] = count
    for i in range(len( dataFrame[columnName])):
        dataFrame.loc[i , columnName] =  indexed[dataFrame[columnName][i]] 

In [22]:
def MakeUniqueInt (dataFrame , columnName):
    maxvalue = np.max(dataFrame[columnName])
    minvalue = np.min(dataFrame[columnName])
    lenghtEachSlice = (maxvalue - minvalue + 1 ) / len(dataFrame[columnName])
    indexed = {}
    for count , value in enumerate( range(len(dataFrame[columnName] / lenghtEachSlice)) ):
        indexed[(value * lenghtEachSlice + minvalue , (value + 1) * lenghtEachSlice + minvalue)] = count
    for i in range(len(dataFrame[columnName])):
        temp = dataFrame.loc[i , columnName]
        flag = False
        for j in indexed :
            if temp >= j[0] and  temp < j[1]:
                flag = True
                dataFrame.loc[i , columnName] = indexed[j]
        if flag == False:
            if temp < indexed[0][0] :
                dataFrame.loc[i , columnName] = -1
            else:
                dataFrame.loc[i , columnName] = len(indexed) + 1

test the MakeUnique

In [11]:
MakeUnique(dataset , "Type")

0      French
1        Thai
2      Burger
3        Thai
4      French
5     Italian
6      Burger
7        Thai
8      Burger
9     Italian
10       Thai
11     Burger
Name: Type, dtype: object
['French' 'Thai' 'Burger' 'Italian']


In [12]:
dataset

,ex,Alt,Bar,Fri,Hun,Pat,Price,Rain,Res,Type,Est,Goal
0,x1,Yes,No,No,Yes,Some,$$$,No,Yes,0,"""0 - 10""",1
1,x2,Yes,No,No,Yes,Full,$,No,No,1,"""30 - 60""",0
2,x3,No,Yes,No,No,Some,$,No,No,2,"""0 - 10""",1
3,x4,Yes,No,Yes,Yes,Full,$,Yes,No,1,"""10 - 30""",1
4,x5,Yes,No,Yes,No,Full,$$$,No,Yes,0,"""> 60""",0
5,x6,No,Yes,No,Yes,Some,$$,Yes,Yes,3,"""0 - 10""",1
6,x7,No,Yes,No,No,None,$,Yes,No,2,"""0 - 10""",0
7,x8,No,No,No,Yes,Some,$$,Yes,Yes,1,"""0 - 10""",1
8,x9,No,Yes,Yes,No,Full,$,Yes,No,2,"""> 60""",0
9,x10,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,3,"""10 - 30""",0


In [42]:
type(dataset["Fri"][1])

str

test the make unique for ints

In [14]:
intlist = [12 , 1 , 123, 23, 22, 5, 77]
dftest = pd.DataFrame(intlist , columns=["test"])


In [23]:
MakeUniqueInt(dftest , "test")

In [24]:
dftest

,test
0,0
1,0
2,6
3,1
4,1
5,0
6,4
